In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


# Choosing THE DATASET

In [21]:
#IMDB Movie Review Dataset
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# DATA PROCESSING

# CHECKING NULL VALUE

In [22]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

# LABELING 'sentiment' IN NUMBER

In [23]:
em = df['sentiment'].unique()
em_num = {}
j=0
for i in em:
    em_num[i] = j;
    j=j+1
df['sentiment'] = df['sentiment'].map(em_num)
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,0
1,A wonderful little production. <br /><br />The...,0
2,I thought this was a wonderful way to spend ti...,0
3,Basically there's a family where a little boy ...,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",0


# Converting 'review' column to lower

In [24]:
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'].head(5)

0    one of the other reviewers has mentioned that ...
1    a wonderful little production. <br /><br />the...
2    i thought this was a wonderful way to spend ti...
3    basically there's a family where a little boy ...
4    petter mattei's "love in the time of money" is...
Name: review, dtype: object

# Removing 'punctuation' marks from text

In [25]:
import string

def remove_punc(txt):
  return txt.translate(str.maketrans('','',string.punctuation))
df['review'] = df['review'].apply(remove_punc)
df['review'].head(5)

0    one of the other reviewers has mentioned that ...
1    a wonderful little production br br the filmin...
2    i thought this was a wonderful way to spend ti...
3    basically theres a family where a little boy j...
4    petter matteis love in the time of money is a ...
Name: review, dtype: object

# Removing numbers from 'review' column

In [26]:
def remove_num(txt):
    st = ""
    for i in txt:
        if not i.isdigit():
            st = st+i
    return st
df['review'] = df['review'].apply(remove_num)
df['review'].head(5)

0    one of the other reviewers has mentioned that ...
1    a wonderful little production br br the filmin...
2    i thought this was a wonderful way to spend ti...
3    basically theres a family where a little boy j...
4    petter matteis love in the time of money is a ...
Name: review, dtype: object

# Removing emoji

In [27]:
def remove_emoji(txt):
    st=""
    for i in txt:
        if i.isascii():
            st=st+i
    return st
df['review'] = df['review'].apply(remove_emoji)
df['review'].head(5)

0    one of the other reviewers has mentioned that ...
1    a wonderful little production br br the filmin...
2    i thought this was a wonderful way to spend ti...
3    basically theres a family where a little boy j...
4    petter matteis love in the time of money is a ...
Name: review, dtype: object

# Applying NLP

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [29]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Removing stopwords

In [30]:
stop_words = set(stopwords.words('english'))

def remove_sw(txt):
    words = txt.split()
    cleaned = []
    for i in words:
        if i.lower() not in stop_words:
            cleaned.append(i)
    return ' '.join(cleaned)

df['review'] = df['review'].apply(remove_sw)


In [31]:
df['review'].head(5)

0    one reviewers mentioned watching oz episode yo...
1    wonderful little production br br filming tech...
2    thought wonderful way spend time hot summer we...
3    basically theres family little boy jake thinks...
4    petter matteis love time money visually stunni...
Name: review, dtype: object

# Splitting the dataset 

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review'],df['sentiment'],test_size=0.2)

# Applying Naive-bayes

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB

bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)


nb_model = MultinomialNB()
nb_model.fit(X_train_bow, y_train)
pred_bow = nb_model.predict(X_test_bow)

# Checking Manually


In [34]:
sample_text = "The movie was good and enjoying"

sample_text_bow = bow_vectorizer.transform([sample_text])
prediction = nb_model.predict(sample_text_bow)

if prediction==0:
    print('Positive')
else:
    print('Negative')


Positive


# MODEL EVALUATION

In [35]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix


In [36]:

#Accuracy Checking
acc_nb = accuracy_score(y_test, pred_bow)
print(f"Accuracy using Naive bayes: {acc_nb*100}%")

Accuracy using Naive bayes: 85.27%


In [37]:
#Precison
precision = precision_score(y_test, pred_bow)
print(f"Precision: {precision*100}%")

Precision: 83.3557821820281%


In [38]:
#recall
recall    = recall_score(y_test, pred_bow)
print(f"Recall: {recall*100}%")

Recall: 87.6923076923077%


In [39]:
#f1 score
f1  = f1_score(y_test, pred_bow)
print(f"F1 Score: {f1*100}%")

F1 Score: 85.46907369044095%


In [40]:
#Confusion Matrix
pred_bow = nb_model.predict(X_test_bow)
cm = confusion_matrix(y_test, pred_bow)

print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[4195  865]
 [ 608 4332]]


# Comparing with Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_bow, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [42]:
#Accuracy using Logistic Regression
pred_lr = lr.predict(X_test_bow)
acc = accuracy_score(y_test, pred_lr)
print(f"Accuracy using Logistic Regression: {acc*100}%")

Accuracy using Logistic Regression: 88.68%


In [43]:
# Precision, Recall, F1
precision = precision_score(y_test, pred_lr)
recall    = recall_score(y_test, pred_lr)
f1        = f1_score(y_test, pred_lr)

print(f"Precision: {precision*100}%")
print(f"Recall: {recall*100}%")
print(f"F1 Score: {f1*100}%")

Precision: 88.6050283860503%
Recall: 88.46153846153845%
F1 Score: 88.53322528363047%


In [44]:
# Confusion Matrix
cm = confusion_matrix(y_test, pred_lr)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[4498  562]
 [ 570 4370]]
